## Assignment 2

In [1]:
# Standard imports for ETL work
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime

# Display settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print(f"Pandas version: {pd.__version__}")
print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Setup complete!")


Pandas version: 2.3.3
Current time: 2026-01-28 15:29:01
Setup complete!
